In [1]:
import os
#if not os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
#    raise 'stop'
if not os.path.isdir('d:/flagellar/'):
    deps_path = '/kaggle/usr/lib/flg-packages/'
    !pip install --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt
    !tar xfvz /kaggle/input/ultralytics-for-offline-install/archive.tar.gz
    !pip install --no-index --find-links=./packages ultralytics
    !rm -rf ./packages

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/flagellar/code/core/')
sys.path.append('/kaggle/input/my-flagellar-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import flg_model
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import flg_yolo

fast_mode = False

In [3]:
all_data = fls.load_all_train_data()
np.random.default_rng(seed=0).shuffle(all_data)

# Pick N tomograms with 1 motor and N tomograms with 0 motors as validation set
N=100
n_motors = np.array([len(d.labels) for d in all_data])
inds_zero = np.argwhere(n_motors==0)[:N,0]
inds_one = np.argwhere(n_motors==1)[:N,0]
inds_test = np.concatenate((inds_zero,inds_one))
inds_train = np.setdiff1d(np.arange(len(n_motors)), inds_test)
inds_test.shape, inds_train.shape

train_data = []
for i in inds_train:
    train_data.append(all_data[i])
test_data = []
for i in inds_test:
    test_data.append(all_data[i])
np.random.default_rng(seed=0).shuffle(test_data)
test_data = test_data
if fast_mode:
    train_data = train_data[1:30]
    test_data = test_data[2:4]
len(train_data), len(test_data)

(29, 2)

In [4]:
importlib.reload(flg_yolo)
model = flg_yolo.YOLOModel()
model.seed = 42
model.use_augs = False
model.run_in_parallel = True
if fast_mode: model.n_epochs = 1
model.train(train_data, test_data)
if not fls.env=='kaggle':
    fls.dill_save(fls.model_dir+ "yolo.pickle", model)

Will process approximately 333 slices for training


Processing training motors:   0%|          | 0/37 [00:00<?, ?it/s]

Will process approximately 9 slices for validation


Processing validation motors:   0%|          | 0/1 [00:00<?, ?it/s]


Processing Summary:
- Train set: 18 tomograms, 37 motors, 333 slices
- Validation set: 1 tomograms, 1 motors, 9 slices
- Total: 19 tomograms, 38 motors, 342 slices

Preprocessing Complete:
- Training data: 18 tomograms, 37 motors, 333 slices
- Validation data: 1 tomograms, 1 motors, 9 slices
- Dataset directory: d:/flagellar/temp//yolo_dataset/
- YAML configuration: d:/flagellar/temp//yolo_dataset/dataset.yaml

Ready for YOLO training!
Starting YOLO training process...
Directory status:
- Train images exists: True
- Val images exists: True
- Train labels exists: True
- Val labels exists: True
Found original dataset.yaml at d:/flagellar/temp//yolo_dataset/dataset.yaml
Fixing YAML paths in d:/flagellar/temp//yolo_dataset/dataset.yaml
Created fixed YAML at d:/flagellar/temp/fixed_dataset.yaml with path: d:/flagellar/temp//yolo_dataset/
Using YAML file: d:/flagellar/temp/fixed_dataset.yaml
YAML contents:
names:
  0: motor
path: d:/flagellar/temp//yolo_dataset/
train: images/train
val: ima

...

Plotting labels to d:\flagellar\temp\yolo_weights\motor_detector\labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to d:\flagellar\temp\yolo_weights\motor_detector
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size




                Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0

                   all          9          9          0          0          0          0

1 epochs completed in 0.002 hours.
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\last.pt, 52.0MB
Optimizer stripped from d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt, 52.0MB

Validating d:\flagellar\temp\yolo_weights\motor_detector\weights\best.pt...
Ultralytics 8.3.98  Python-3.10.14 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti, 12282MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs



                Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<0

                   all          9          9          0          0          0          0
Speed: 0.2ms preprocess, 16.0ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to d:\flagellar\temp\yolo_weights\motor_detector

Training complete!


In [5]:
if not fls.is_submission:
    importlib.reload(flg_yolo)    
    inferred_test_data = model.infer(test_data)
    print(fls.score_competition_metric(inferred_test_data, test_data))

0.0


In [6]:
inferred_test_data2 = model.infer(fls.load_all_test_data())
fls.write_submission_file(inferred_test_data2)

       tomo_id  Motor axis 0  Motor axis 1  Motor axis 2
0  tomo_003acc            -1            -1            -1
1  tomo_00e047            -1            -1            -1
2  tomo_01a877            -1            -1            -1


In [7]:
inferred_test_data2

[Data(is_train=False, name='tomo_003acc', labels=Empty DataFrame
 Columns: []
 Index: [], labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_shape=(0, 0, 0), voxel_spacing=nan, mean_per_slice=array([], dtype=float64), std_per_slice=array([], dtype=float64), percentiles_per_slice=array([], shape=(101, 0), dtype=float64)),
 Data(is_train=False, name='tomo_00e047', labels=Empty DataFrame
 Columns: []
 Index: [], labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_shape=(0, 0, 0), voxel_spacing=nan, mean_per_slice=array([], dtype=float64), std_per_slice=array([], dtype=float64), percentiles_per_slice=array([], shape=(101, 0), dtype=float64)),
 Data(is_train=False, name='tomo_01a877', labels=Empty DataFrame
 Columns: []
 Index: [], labels_unfiltered=Empty DataFrame
 Columns: []
 Index: [], loaded_state='unloaded', data=None, data_shape=(0, 0, 0), voxel_spacing=nan, mean_per_slice=array([], dtype

In [9]:
import ultralytics
ultralytics.YOLO('yolov8m.yaml')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_